In [15]:
import open3d as o3d 
import numpy as np
from pathlib import Path
import scipy
import math
from tqdm import tqdm

In [16]:
# get data base path to collect the point clouds
BASE = Path().resolve().parents[1]
DATA_PATH = BASE / 'data'
print(DATA_PATH)
SHOW_PC = True
THRESHOLD = 0.5  # 5 cm
COLORS = {'green': [0, 255, 0], 'red': [255, 0, 0], 'blue': [0, 0, 255],
          'black': [0, 0, 0], 'yellow': [255, 255, 0]}

/home/dayoff/codes/point_cloud_lib/data


In [17]:
def calc_eucl_dist(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

In [18]:
def check_threshold(p1, p2):
    if ((p2[0] + THRESHOLD > p1[0]) and (p1[0] > p2[0] - THRESHOLD)) and \
        ((p2[1] + THRESHOLD > p1[1]) and (p1[1] > p2[1] - THRESHOLD)) and \
        ((p2[2] + THRESHOLD > p1[2]) and (p1[2] > p2[2] - THRESHOLD)):
        return calc_eucl_dist(p1, p2) <= THRESHOLD

In [19]:
def load_point_cloud(idx, color='blue'):
    # get first point cloud, assign it with a blue color
    filename = f'{idx}'.zfill(10) + '.ply'
    pc_path = DATA_PATH / 'kitti' / filename
    print('Collecting pc: ', pc_path)
    pcd = o3d.io.read_point_cloud(str(pc_path), format='ply')
    pcd_size = len(np.asarray(pcd.points))

    # define point cloud color
    np_colors = np.array([COLORS[color] for _ in range(pcd_size)])
    pcd.colors = o3d.utility.Vector3dVector(np_colors)

    pc_tot_size = len(point_clouds)
    # show point cloud info
    print(f'PCD {pc_tot_size}:\n', pcd, f'\nPCD {pc_tot_size} size: ', pcd_size,
          f'\n\nPCD {pc_tot_size} colors: \n', np.asarray(pcd.colors),
          f'\n\nPCD {pc_tot_size} points:\n', np.asarray(pcd.points))
    return pcd

In [20]:
def show_pc(pc_list):
    if SHOW_PC:
        new_pc_list = []
        pc_1 = pc_list[0]
        # define first point cloud color
        pc_1_size = len(np.asarray(pc_1.points))
        np_colors = np.array([COLORS['red'] for _ in range(pc_1_size)])
        pc_1.colors = o3d.utility.Vector3dVector(np_colors)
        new_pc_list.append(pc_1)

        if len(pc_list) == 2 or len(pc_list) == 3:
            pc_2 = pc_list[1]

            # offset the second point cloud
            offset_pc = o3d.geometry.PointCloud(pc_list[1])
            offset_pc.points = o3d.utility.Vector3dVector(
                np.asarray(pc_2.points) + [0.0, 0.0, -5.0])
            offset_pc_size = len(np.asarray(offset_pc.points))

            # define second point cloud color
            np_colors = np.array([COLORS['green'] for _ in range(offset_pc_size)])
            offset_pc.colors = o3d.utility.Vector3dVector(np_colors)
            
            new_pc_list.append(offset_pc)
        
        if len(pc_list) == 3:
            pc_3 = pc_list[2]

            # offset the third point cloud
            offset_pc = o3d.geometry.PointCloud(pc_3)
            offset_pc.points = o3d.utility.Vector3dVector(
                np.asarray(pc_3.points) + [0.0, 0.0, -10.0])
            offset_pc_size = len(np.asarray(offset_pc.points))

            # define the point cloud color
            np_colors = np.array([COLORS['blue'] for _ in range(offset_pc_size)])
            offset_pc.colors = o3d.utility.Vector3dVector(np_colors)

            new_pc_list.append(offset_pc)

        o3d.visualization.draw_geometries(new_pc_list)

In [21]:
def sort_pc(pc):
    pc_np = np.asarray(pc.points)
    idx_sorted_pc = np.lexsort((pc_np[:,1], pc_np[:,0]))
    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(pc_np[idx_sorted_pc])
    return new_pc

In [22]:
def get_min_max(pc):
    pc_np = np.asarray(pc.points)
    p_max, p_min = np.amax(pc_np, axis=0), np.amin(pc_np, axis=0)
    return p_max, p_min

In [23]:
def split_map(pc):
    pc_np = np.asarray(pc.points)
    new_pc_np = pc_np[(pc_np[:,0] >= -1.5) & (pc_np[:,0] <= 1.5)]
    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(new_pc_np)
    print(f'Reduzed {len(pc_np)} point to {len(new_pc_np)} points.')
    return new_pc

In [24]:
def brute_force_eucl_dist(pc_1, pc_2):  # too slow
    pts_set = set()
    for idx_1, p1 in enumerate(tqdm(np.asarray(pc_1.points))):
        for idx_2, p2 in enumerate(np.asarray(pc_2.points)):
            if check_threshold(p1, p2):
                pts_set.add(idx_1)
    return pts_set

In [25]:
def get_pc_from_pts_set(pc_set, pc):
    pts_list = list(pc_set)
    pc_np = np.asarray(pc.points) 
    pc_np_from_pts = pc_np[pts_list]

    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(pc_np_from_pts)
    return new_pc

In [26]:
def scipy_eucl_dist(pc_1, pc_2):
    dist = scipy.spatial.distance.cdist(np.asarray(pc_1.points), np.asarray(pc_2.points))

In [27]:
point_clouds = []

In [30]:
pc = load_point_cloud(0, 'blue')
sorted_pc = sort_pc(pc)
partial_pc = split_map(sorted_pc)
point_clouds.append(partial_pc)
show_pc([pc, partial_pc])

PCD 2:
 geometry::PointCloud with 120574 points. 
PCD 2 size:  120574 

PCD 2 colors: 
 [[  0.   0. 255.]
 [  0.   0. 255.]
 [  0.   0. 255.]
 ...
 [  0.   0. 255.]
 [  0.   0. 255.]
 [  0.   0. 255.]] 

PCD 2 points:
 [[74.536  9.937  2.752]
 [74.558 10.178  2.754]
 [74.569 10.419  2.755]
 ...
 [ 3.705 -1.394 -1.73 ]
 [ 3.611 -1.345 -1.681]
 [ 3.73  -1.377 -1.738]]
Reduzed 120574 point to 14467 points.


In [29]:
pc = load_point_cloud(10, 'red')
sorted_pc = sort_pc(pc)
partial_pc = split_map(sorted_pc)
point_clouds.append(partial_pc)
show_pc([pc, partial_pc])

PCD 1:
 geometry::PointCloud with 122227 points. 
PCD 1 size:  122227 

PCD 1 colors: 
 [[255.   0.   0.]
 [255.   0.   0.]
 [255.   0.   0.]
 ...
 [255.   0.   0.]
 [255.   0.   0.]
 [255.   0.   0.]] 

PCD 1 points:
 [[78.601 10.984  2.893]
 [67.496  9.644  2.515]
 [67.487  9.859  2.515]
 ...
 [ 3.779 -1.43  -1.768]
 [ 3.809 -1.427 -1.78 ]
 [ 3.786 -1.405 -1.767]]
Reduzed 122227 point to 14497 points.


In [127]:
cluster_pts = brute_force_eucl_dist(point_clouds[0], point_clouds[1])

100%|██████████| 14467/14467 [05:32<00:00, 43.51it/s]


In [130]:
cluster_pc = get_pc_from_pts_set(cluster_pts, point_clouds[0])

In [143]:
show_pc([point_clouds[0], point_clouds[1], cluster_pc])